# Federal University of Ceará
# Teleinformatics Departament
# Graduate Program in Teleinformatics Engeneering
## TIP8419 - Tensor Algebra
## Homework 9 - Alternating Least Squares (ALS) Algorithm
### Report and Simulation results

- Ezequias Márcio - 497779

To run this notebook properly, it is necessary Python3 installed alongside with the packages listed below:

- `numpy 1.17.2`
- `scipy 1.4.1`
- `tdqm 4.36.1`
- `bokeh 1.3.4`

Make sure that the files `tensoralg.py` and `ta_simulations.py` are in the same directory as this notebook. In this files, it can be found the tensor algebra module functions and the code listings of the simulations.

In [2]:
# Importing the simulation module:
from ta_simulations import *
np.set_printoptions(4, linewidth=175)
output_notebook()
# Loading the .m file given as a python dictionary:
cpd_data = loadmat('files/cpd_tensor.mat')

Loading BokehJS ...

### Problem 1

For the third-order tensor $\mathbf{\mathcal{X}} \in \mathbb{C}^{I \times J \times K}$ provided in the file `cpd_tensor.mat`, implement the plain-vanilla Alternating Least Squares (ALS) algorithm that estimates the factor matrices $\mathbf{A} \in \mathbb{C}^{I \times R}$ , $\mathbf{B} \in \mathbb{C}^{J \times R}$ and $\mathbf{C} \in \mathbb{C}^{K \times R}$ by solving the following problem:

\begin{equation}
    (\hat{\mathbf{A}}, \hat{\mathbf{B}}, \hat{\mathbf{C}}) = \underset{\mathbf{A}, \mathbf{B}, \mathbf{C}}{\text{min}} || \mathbf{\mathcal{X}} - \sum^{R}_{r = 1} \mathbf{a}_{r} \circ \mathbf{b}_{r} \circ \mathbf{c}_{r} ||^{2}_{F}
\end{equation}

where $A = [a_{1} , . . . , a_{R}]$, $B = [b_{1} , . . . , b_{R}]$ and $C = [c_{1} , . . . , c_{R}]$. Considering a successful run,
compare the estimated matrices $\hat{\mathbf{A}}$, $\hat{\mathbf{B}}$ and $\hat{\mathbf{C}}$ with the original ones. Explain the results.

### Solution:

Using the provided data in `cpd_tensor.mat`, it can be seen below that the implemented algorithm is working as expected. It was performed the decomposition of the tensor X and computed the normalized mean squared error between the estimated factor matrices and the real ones, and the error between the orininal tensor and the estimative.

In [3]:
# Validating the results with the data provided:
a_test = cpd_data['A']; b_test = cpd_data['B']; c_test = cpd_data['C']; tx_test = cpd_data['tenX'].transpose(2,0,1) 
matrices = [a_test, b_test, c_test]
print(f'Tensor X {tx_test.shape}\nMatrix A {a_test.shape}\nMatrix B {b_test.shape}\nMatrix C {c_test.shape}\n\nApplying ALS:')
# Estimating the factor matrices A, B and C via ALS considering eps=1e-6 and 500 iterations:
rank = a_test.shape[1]; matrices_hat, als_error = tensoralg.cp_decomp(tx_test, rank, verb=True)
tx_hat = tensoralg.cpd_tensor(matrices_hat) # Using the estimated matrices to reconstruct X:
# Plotting the error between iteractions:
plot_error(als_error)
# Calculating the Normalized Mean Squared Error:
nmse_m = [norm_mse(matrices[n], matrices_hat[n]) for n in range(len(matrices))]; nmse_X = norm_mse(tx_test, tx_hat)
print(f'''Normalized Mean Squared Errors:\n- Factor matrices A, B, C: {nmse_m}\n- Tensor X: {nmse_X}''')

Tensor X (5, 8, 4)
Matrix A (8, 3)
Matrix B (4, 3)
Matrix C (5, 3)

Applying ALS:
Algorithm converged with 50 iterations and an achieved error of [9.5642e-07] between iteractions.


Normalized Mean Squared Errors:
- Factor matrices A, B, C: [2.565650598586946, 2.5086346132228545, 1.2456471389124377]
- Tensor X: 2.3272272565384376e-09


Additionally, the error between iterations of the ALS algorithm is shown in the plot above to ilustrate de performance of the algorithm for this run.

### Problem 2

Assuming 1000 Monte Carlo experiments, generate a tensor $\mathbf{\mathcal{X}_{0}} = \text{CPD}(\mathbf{A},\mathbf{B},\mathbf{C})$ where $\mathbf{A} \in \mathbb{C}^{I \times R}$ , $\mathbf{B} \in \mathbb{C}^{J \times R}$ and $\mathbf{C} \in \mathbb{C}^{K \times R}$ have unit norm columns with elements randomly drawn from a Normal distribution, with R = 3. Let $\mathbf{\mathcal{X}} = \mathbf{\mathcal{X}_{0}} + \alpha \mathbf{\mathcal{V}}$ be a noisy version of $\mathbf{\mathcal{X}}$ where $\mathbf{\mathcal{V}}$ is the additive noise term, whose elements are drawn from a normal distribution. The parameter $\alpha$ controls the power (variance) of the noise term, and is defined as a function of the signal to noise ratio (SNR), in dB, as follows:

\begin{equation}
    \text{SNR}_{dB} = 10 \log_{10} \frac{||\mathbf{\mathcal{X}_{0}}||^{2}_{F}}{||\alpha \mathbf{\mathcal{V}}||^{2}_{F}}
\end{equation}

Assuming the SNR range $[0, 5, 10, 15, 20, 25, 30]$ dB, find the estimates $\hat{\mathbf{A}}$, $\hat{\mathbf{B}}$ and $\hat{\mathbf{C}}$ obtained with the ALS algorithm for the configurations $(I,J,K) = (10,4,2)$.

Let us define the normalized mean square error (NMSE) measure as follows:

\begin{equation}
    \text{NMSE}(\mathbf{X_{0}}) = \frac{1}{1000} \sum^{1000}_{i = 1} \frac{||\hat{\mathbf{\mathcal{Q}_{0}(i)}} - \mathbf{\mathcal{Q}_{0}(i)}||^{2}_{F}}{||\mathbf{\mathcal{Q}_{0}(i)}||^{2}_{F}}
\end{equation}

where $\mathbf{\mathcal{Q}_{0}(i)}$ and $\hat{\mathbf{\mathcal{Q}_{0}(i)}}$ represent the original data matrix and the reconstructed one at the ith Monte Carlo experiment, respectively. For each SNR value and configuration, plot the NMSE(A), NMSE(B) and NMSE(C) as a function of the SNR. Discuss the obtained results.

### Solution:

The simulation results are generated in the cell below. First, for the 1000 realizations, the tensor $\mathcal{X}_{0}(i)$ is generated and then, for each value of SNR, it is added Gaussian noise to the tensor that will be the input of the ALS algorithm. After that, the estimated factor matrices $\hat{\mathbf{A}}, \hat{\mathbf{B}}$ and $\hat{\mathbf{C}}$ are used to build the estimative $\hat{\mathcal{X}}_{0}(i)$. Lastly, the NMSE between the estimated factor matrices and the original and the recontructed tensor are sorted. This process is implemented in the function `run_simulation_als`.

In [4]:
rows = [10, 4, 2]; rank = 3 # Factor matrices shape: I,R; J,R and K,R
snr = np.linspace(0, 30, 7)  # SNR values
mc_realizations = 100       # Monte Carlo Realizations
# Generating results: uncomment for generate new data
# result = run_simulation_als(snr, mc_realizations, rows, rank, 1e-8, 1000)
# result2 = run_simulation_als(snr, mc_realizations, rows, rank, 1e-6, 500)
# Simulations take several minuntes, the results are saved in the directory 'files/result-als*'

The simulations take several minutes to run, so the results were generate offline and loaded in the cell below. It was considered 2 scenarios for the Monte Carlo experiments:
 
- Case 1: ALS algorithm with $\delta = 10^{-8}$, considering 1000 iterations;
- Case 2: ALS algorithm with $\delta = 10^{-6}$, considering 500 iterations.

In [5]:
# Loading simulation results:
result = [np.load('files/result-als-1e8-1000it/resulta.npy'), 
          np.load('files/result-als-1e8-1000it/resultb.npy'),
          np.load('files/result-als-1e8-1000it/resultc.npy'), 
          np.load('files/result-als-1e8-1000it/resultx.npy')]

result2 = [np.load('files/result-als-1e6-500it/result2a.npy'), 
           np.load('files/result-als-1e6-500it/result2b.npy'), 
           np.load('files/result-als-1e6-500it/result2c.npy'), 
           np.load('files/result-als-1e6-500it/result2x.npy')]

labels = ['NMSE A','NMSE B','NMSE C', 'NMSE X']

In [10]:
plot_results_als(snr, result, 'ALS', labels) # Case 1

In [11]:
plot_results_als(snr, result2, 'ALS', labels) # Case 2

With respect to the plots above, it can be seen that considering more iteractions and more precision ($\delta$), the NMSE between the tensor and his estimative reaches smaller values up to $10^{-3}$.